# ALBERT – A Lite BERT

Transformer models, especially BERT transformed the NLP pipeline. They solved the problem of sparse annotations for text data. Instead of training a model from scratch, we can now simply fine-tune existing pre-trained models. But the sheer size of BERT(340M parameters) makes it a bit unapproachable. It is very compute-intensive and time taking to run inference using BERT.ALBERT is a lite version of BERT which shrinks down the BERT in size while maintaining the performance.

This model was published in a paper presented at ICLR 2020 by Zhenzhong Lan, Mingda Chen2, Sebastian Goodman, Kevin Gimpel, Piyush Sharma and Radu Soricut (researchers at Google Research and Toyota Technological Institute at Chicago).


The main Idea of ALBERT is to reduce the number of parameters(up to 90% reduction) using novel techniques while not taking a big hit to the performance. Now, this compressed version scales a lot better than the original BERT, improving the performance while still keeping the model small.

To read about it more, please refer [this](https://analyticsindiamag.com/complete-guide-to-albert-a-lite-bertwith-python-code/) article.

# Usage of Albert

Tensorflow hub has made it extremely easy to use pre-trained models. Let’s use pretrained ALBERT base model for the classification of movie reviews.

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras nltk gensim simpletransformers --user -q --no-warn-script-location


In [ ]:
!python -m pip install sentencepiece --user -q --no-warn-script-location
!python -m pip install tensorflow_text --user -q --no-warn-script-location

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text 
import tensorflow_datasets as tfds

In [ ]:
albert_url='https://tfhub.dev/tensorflow/albert_en_base/2'
encoder = hub.KerasLayer(albert_url)

preprocessor_url="https://tfhub.dev/tensorflow/albert_en_preprocess/3"
preprocessor = hub.KerasLayer(preprocessor_url)

Model and the required preprocessor are downloaded and loaded.

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)

In [ ]:
encoder_inputs

In [ ]:
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]     
sequence_output = outputs["sequence_output"]
pooled_output

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [ ]:
embedding_model = tf.keras.Model(text_input, pooled_output)

Just like that, we have our embedding layer. We just need to build a Fully connected neural network to predict whether a movie review is positive or negative.

In [ ]:
model = tf.keras.Sequential()
model.add(embedding_model)

model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(128),
                    epochs=100,
                    validation_data=validation_data.batch(128),
                    verbose=1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
font = {'family' : 'normal',
        'size'   : 22}
plt.tight_layout()
matplotlib.rc('font', **font)
plt.figure(figsize=(15,8))
plt.plot(history.history['accuracy'],label='aacuracy')
plt.plot(history.history['val_accuracy'],label='validation_accuracy')
plt.legend(bbox_to_anchor=(0.7, 0.7))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('foo1.png')
plt.show()

With this basic model validation accuracy, about 75% is a good number. Especially when we are not fine-tuning the embeddings at all. We can fine-tune the embeddings by just making the encoder trainable.

In [ ]:
albert_url='https://tfhub.dev/tensorflow/albert_en_base/2'
encoder = hub.KerasLayer(albert_url,trainable=True)

preprocessor_url="https://tfhub.dev/tensorflow/albert_en_preprocess/3"
preprocessor = hub.KerasLayer(preprocessor_url)

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
encoder_inputs = preprocessor(text_input)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]     
sequence_output = outputs["sequence_output"]
embedding_model = tf.keras.Model(text_input, pooled_output)
model2 = tf.keras.Sequential()
model2.add(embedding_model)
model2.add(tf.keras.layers.Dense(128, activation='relu'))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dense(30, activation='relu'))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dense(8, activation='relu'))
model2.add(tf.keras.layers.BatchNormalization())
model2.add(tf.keras.layers.Dense(1))
model2.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
history2 = model2.fit(train_data.shuffle(10000).batch(128),
                    epochs=10,
                    validation_data=validation_data.batch(128),
                    verbose=1)

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(history.history['accuracy'],label='accuracy')
plt.plot(history.history['val_accuracy'],label='validation_accuracy')
plt.plot(history2.history['accuracy'],label='accuracy_finetuned')
plt.plot(history2.history['val_accuracy'],label='validation_accuracy_finetuned')
plt.legend(bbox_to_anchor=(0.7, 0.7))
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.savefig('foo2.png')
plt.show()